* setting up a generic Kubernetes system
* deploying Laravel onto it using Helm charts (versionable infrastructure description), with Postgres, Redis and workers
* employing a Gitlab repository for CI/CD for building container images
* managing per-deployment configuration and environment variables
* process health, logging and scaling
* using the artisan CLI tool for scheduled and one-off jobs on the cluster (as time permits)

In [21]:
./setup_account.sh
export USER=$GIT_COMMITTER

# Kubernetes Workshop

With Phil Weir
(the Belfast guy who is somewhere near this projector, hopefully)

* Structure of course
* Next steps

Initial set-up: _15min_ (-> 0:15)

### Why Kubernetes?

_10 min_ (-> 0:25)

* How is this relevant to PHP?
  * Scalable
  * Systematic
  * Within control

* Why Kubernetes over alternatives?
  * Other container orchestrators
    * Popularity and backing
    * Good balance of simplicity and complexity
  * IaaS/PaaS
    * Doesn't tie you to a single provider
    * Can be run off-cloud
    * Can be run on a dev machine
    * Abstracts and automates the provider-specific bits (mostly)
    * Separates out hardware from application infrastructure (containers vs 

* Brief (re)introduction to Docker
  * If you aren't already familiar with it, you can think of a docker _container_ as a VM that shares a kernel with the host
    * ...but don't as that loses key important differences
  * Recommended practice is to have one process per container
    * containers are very memory light compared to VMs, so this is much less wasteful than it may sound
    * this provides encapsulation and makes scaling easier
  * `docker-compose` is an _extremely_ handy tool that takes a short, app-specific _docker-compose.yml_ file as input and spins up a multi-container environment with all the expected dependencies and links
    * you can keep the `docker-compose.yml` file in the repo with your app
    * between docker and docker-compose, you can provide a rough alternative to VirtualBox and Vagrant developer flow (although those do not map directly)

### Setting up a system

_5 min_ (-> 0:30)

* Cloud
  * GKE: gcloud
  * AWS: EKS
  * Azure: AKS

* Manually (kubeadm / Kubernetes the Hard Way --->)
  * kops
  * kubeadm
  * Kubernetes the Hard Way

* Minikube...
  * Those who want to follow along, please do
  * Those who are happy to use the online system, please do

_15 min_ (-> 0:45)

[GETTING STARTED - kubectl, minikube, git, docker-compose and docker]

In [3]:
git version

git version 2.17.1


In [4]:
kubectl version

Client Version: version.Info{Major:"1", Minor:"11", GitVersion:"v1.11.2", GitCommit:"bb9ffb1654d4a729bb4cec18ff088eacc153c239", GitTreeState:"clean", BuildDate:"2018-08-07T23:17:28Z", GoVersion:"go1.10.3", Compiler:"gc", Platform:"linux/amd64"}
Server Version: version.Info{Major:"1", Minor:"10+", GitVersion:"v1.10.6-gke.4", GitCommit:"39a7d039cc0b45c80f755125d0506e44555ed405", GitTreeState:"clean", BuildDate:"2018-09-13T22:15:39Z", GoVersion:"go1.9.3b4", Compiler:"gc", Platform:"linux/amd64"}


To begin with, I will talk a little bit about basic Kubernetes tools and concepts, then we can start building up practical steps...

The language of Kubernetes communication is JSON, under the hood, but generally the tooling lets you talk to it in YAML, which is actually a (much more readable) superset. Instead of

```json
{
    "apiVersion": "v1",
    "type": "Pod",
    "metadata": {
        "name": "hi-pod"
    }
    ...
}
```

we can write

```yaml
apiVersion: v1
type: Pod
metadata:
    name: "hi-pod"
```

We can build up all our Kubernetes objects on the cluster by sending declarative YAML files.

# Step 1: Just nginx

The easiest way to interact with a Kubernetes cluster is the `kubectl` tool. It's preinstalled here, but you can download it yourself locally. The configuration files are called `kubeconfig` files, and the default one lives at `~/.kube/config` on Mac/*nix.

In [5]:
kubectl run mynginx --image=nginx

deployment.apps/mynginx created


This `run` command is not so common day-to-day, but is an opinionated shortcut, bundling a couple of more common steps, to get a Docker image running on a cluster as a standalone application or "deployment".

A _deployment_ is essentially a single application (perhaps running many times). It is normally one or more replicas of a specific process (in this case nginx), maybe with some helper process or some start-up/shutdown actions.

In [6]:
kubectl get deployments

NAME      DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
mynginx   1         1         1            0           3s


We can run two identical instances of nginx, by scaling it

In [7]:
kubectl scale deployment mynginx --replicas=2
kubectl get deployments

deployment.extensions/mynginx scaled
NAME      DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
mynginx   2         2         2            1           2m


However, for us to interact with nginx we have to encounter another concept: _services_ . Traditionally in the PHP ecosystem, we often think of a process and the service it provides as essentially equivalent - however, modern orchestration tools help decouple the idea of an advertised service (_services_) and the processes backing them up (_deployments_).

This means that we can link up different tools with them only knowing each others' services - Kubernetes will route requests through to the deployed processes behind the scenes, with simple load balancing. For example, there could be 100 nginx processes spread over a dozen VMs and any internal or external request just asks for the nginx service and magically gets a reply.

We still need to create a service though... as with `run` for deployments, there is a command to bundle a couple of steps: `expose`

In [8]:
kubectl expose deployment mynginx --port=80
kubectl get services

service/mynginx exposed
NAME      TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)   AGE
mynginx   ClusterIP   10.15.254.252   <none>        80/TCP    1s


There are certain service types that involve Kubernetes creating and attaching an external load balancer on the cloud platform - so, for instance, you can tell an AWS-based Kubernetes cluster that you want an external load balancer to point to nginx, and it will tell AWS to fire one up and show the `EXTERNAL IP` in this list, without you having to plumb it in. Your nginx processes will then be available publicly (by default).

This is a short workshop to cover a lot of concepts, but lets take a brief break to check that works:

In [9]:
IP=$(kubectl get service mynginx --output=jsonpath="{.spec.clusterIP}")
curl http://$IP

<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
    body {
        width: 35em;
        margin: 0 auto;
        font-family: Tahoma, Verdana, Arial, sans-serif;
    }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>


[In another notebook](/user/philtweir/notebooks/kubernetes-scotlandphp/visual-notebook.ipynb), there is a bit of Python for showing a snippet view of the rendered page. We will use this further later also, so do open it in a new tab and check it works.

To keep things simple and consistent, we will use private IPs - so we can't navigate to the public URL in a browser, and instead show the output in a notebook - but the jump to public on a cloud provider is also pretty straightforward.

One final bit of machinery we should see before we move on:

In [10]:
kubectl get pods

NAME                       READY     STATUS    RESTARTS   AGE
mynginx-7f77c9fb4c-9lmbh   1/1       Running   0          1m
mynginx-7f77c9fb4c-r9sfz   1/1       Running   0          3m


_Pods_ are the individual processes running on a machine somewhere. Each of those is an nginx process. They are the basic unit of Kubernetes execution - here, two Pods were created as part of our Deployment (remember, we scaled it to 2 processes)

Strictly, pods _can_ be more than one process. They are usually, but not always one Docker container (which is _generally_ one OS process) - however, in some cases a "sidecar" container is useful. For instance, a postgres metric-exporter process running alongside the postgres database process, that sends resource usage stats. You see the `1/1` appearing under `READY` above - this means that all 1 of the 1 containers in the Pod are good to go.

In any case, conceptually, a Pod still represents one instance of one tool. They are created and destroyed as part of Deployments, Jobs, and many other Kubernetes objects.

Finally, we tidy up:

In [11]:
kubectl delete deployment mynginx
kubectl delete service mynginx

deployment.extensions "mynginx" deleted
service "mynginx" deleted


In [12]:
kubectl get deployment

No resources found.


In [14]:
kubectl get pods

NAME                       READY     STATUS        RESTARTS   AGE
mynginx-7f77c9fb4c-r9sfz   0/1       Terminating   0          3m


All gone! (you might need to try a couple of times, while you wait for it to terminate) As the Deployment was deleted, the pods that made it up, went away too.

These are only a few of the Kubernetes concepts and objects, and the above is an intentionally hand-wavey introduction, to give you a feel for some key components. For the moment, we'll steer back to PHP.

### Local Development

_20 min_ (-> 1:05)

* docker-compose
  * like orchestration-lite
  * ties in closely with docker-swarm, but equally useful standalone
  * allows your process and connections to be described in one file
  * helps you manage variables, linking and persistance
  * great test-bed

* Buckram
  * an evolving set of git-based configuration templates
  * one of many options, helping to go from source repo to dockerized, deployable app

[Go through key points]

# Step 2: Bare LAMP

_25 min_ (-> 1:30)

* Setting up Helm
  * Helm describes itself as a Kubernetes package manager
  * What this means is that a whole set of interlinking components, secrets, services can be described under one banner and jointly deployed as a "Chart"
  * This could be a full app, or a self-backing-up database, or monitoring service

When setting Helm up on a new cluster, you should run `helm init`

In [40]:
export TILLER_NAMESPACE=$GIT_COMMITTER_NAME
helm init --service-account jupyter-user

$HELM_HOME has been configured at /home/jovyan/.helm.
(Use --client-only to suppress this message, or --upgrade to upgrade Tiller to the current version.)
Happy Helming!


This sets up a daemon on the cluster, called `tiller` (you can sea the theme...), which manages the actual deployment. The local `helm` binary mostly communicates with this daemon.

Helm can tell us what charts are installed already:

In [41]:
helm list

Without getting too much into all the `helm` functionality, let's start by seeing how we can deploy an nginx server to our Kubernetes cluster. Helm uses Kubernetes libraries under the hood, so your default authentication configuration (from ~/.kube/config) will get used.

In [42]:
helm repo update
helm search php

Hang tight while we grab the latest from your chart repositories...
...Skip local chart repository
...Successfully got an update from the "stable" chart repository
Update Complete. ⎈ Happy Helming!⎈ 
NAME             	CHART VERSION	APP VERSION            	DESCRIPTION                                       
stable/phpbb     	3.0.2        	3.2.3                  	Community forum that supports the notion of use...
stable/phpmyadmin	1.1.1        	4.8.2                  	phpMyAdmin is an mysql administration frontend    
stable/joomla    	3.0.2        	3.8.12                 	PHP content management system (CMS) for publish...
stable/lamp      	0.1.5        	5.7                    	Modular and transparent LAMP stack chart suppor...
stable/mediawiki 	4.0.3        	1.31.1                 	Extremely powerful, scalable software and a fea...
stable/osclass   	3.0.2        	3.7.4                  	Osclass is a php script that allows you to quic...
stable/dokuwiki  	3.0.2        	0.20180422.20180503

In [43]:
helm install stable/lamp --name mylamp

NAME:   mylamp
LAST DEPLOYED: Wed Oct  3 22:30:57 2018
NAMESPACE: scotlandphp-XXXXX
STATUS: DEPLOYED

RESOURCES:
==> v1/Secret
NAME         TYPE    DATA  AGE
mylamp-lamp  Opaque  0     0s

==> v1/ConfigMap
NAME               DATA  AGE
mylamp-lamp-httpd  3     0s
mylamp-lamp-php    2     0s

==> v1/PersistentVolumeClaim
NAME         STATUS   VOLUME    CAPACITY  ACCESS MODES  STORAGECLASS  AGE
mylamp-lamp  Pending  standard  0s

==> v1/Service
NAME         TYPE          CLUSTER-IP    EXTERNAL-IP  PORT(S)       AGE
mylamp-lamp  LoadBalancer  10.15.252.28  <pending>    80:32431/TCP  0s

==> v1beta1/Deployment
NAME         DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
mylamp-lamp  1        1        1           0          0s

==> v1/Pod(related)
NAME                         READY  STATUS   RESTARTS  AGE
mylamp-lamp-ff75d949d-v5nk7  0/2    Pending  0         0s


NOTES:
INIT:
      Please wait for all init containers to finish before connecting to
      the charts services. This might take a f

You can see from the lines starting with `==>` that a whole series of objects have been created, including a Deployment and a Service. As we learned above, a Deployment will create one or more Pods - you can see the Pod starts with the same name and is marked "related".

In [47]:
kubectl get services
kubectl get pods

NAME            TYPE           CLUSTER-IP      EXTERNAL-IP     PORT(S)        AGE
mylamp-lamp     LoadBalancer   10.15.252.28    35.197.238.10   80:32431/TCP   1m
tiller-deploy   ClusterIP      10.15.252.217   <none>          44134/TCP      5m
NAME                            READY     STATUS    RESTARTS   AGE
mylamp-lamp-ff75d949d-v5nk7     2/2       Running   0          1m
tiller-deploy-d8db78b5b-68spl   1/1       Running   0          3m


In [48]:
IP=$(kubectl get service mylamp-lamp --output=jsonpath="{.spec.clusterIP}")
curl http://$IP

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>403 Forbidden</title>
</head><body>
<h1>Forbidden</h1>
<p>You don't have permission to access /
on this server.<br />
</p>
</body></html>


Well, this does makes sense - our LAMP server has no particular PHP app set up on it, so there's nothing to serve - rather than giving a meaningless default page when the app is missing, it gives access denied.

In this case, the contributors of the `stable/lamp` Chart expect us to provide custom configuration when deploying, for instance, for a Wordpress or custom app deployment. There's no reason we _have_ to use this chart to do that (there's also a separate Wordpress chart above, for instance). We will show a more complete app, while we explore more of helm.

In [49]:
helm delete --purge mylamp

release "mylamp" deleted


* Concepts of charts and Docker images

So what actually _is_ a Chart?

It is a filetree of, mostly, template [YAML](https://en.wikipedia.org/wiki/YAML) files. Each of these defines a Kubernetes object, such as a Deployment, Service or Secret. This way you can have all the boilerplate in a single git-versioned Chart, dynamically dropping in per-deployment settings using a handful of templated variables at deployment time - such the public URL for nginx, or the back-up retention period for postgres.

In [50]:
rm -rf ~/helm-charts
git clone https://github.com/helm/charts ~/helm-charts

Cloning into '/home/jovyan/helm-charts'...
remote: Enumerating objects: 2, done.        
remote: Counting objects: 100% (2/2), done.        
remote: Compressing objects: 100% (2/2), done.        
remote: Total 40880 (delta 0), reused 1 (delta 0), pack-reused 40878        
Receiving objects: 100% (40880/40880), 12.14 MiB | 18.89 MiB/s, done.
Resolving deltas: 100% (27167/27167), done.


In [51]:
cd ~/helm-charts/stable
ls

acs-engine-autoscaler	hubot			 pachyderm
aerospike		inbucket		 parse
anchore-engine		influxdb		 percona
apm-server		ipfs			 percona-xtradb-cluster
ark			janusgraph		 phabricator
artifactory		jasperreports		 phpbb
artifactory-ha		jenkins			 phpmyadmin
auditbeat		joomla			 postgresql
aws-cluster-autoscaler	k8s-spot-rescheduler	 prestashop
bitcoind		kanister-operator	 presto
bookstack		kapacitor		 prometheus
buildkite		katafygio		 prometheus-adapter
burrow			keel			 prometheus-blackbox-exporter
centrifugo		keycloak		 prometheus-cloudwatch-exporter
cerebro			kiam			 prometheus-mysql-exporter
cert-manager		kibana			 prometheus-node-exporter
chaoskube		kong			 prometheus-postgres-exporter
chartmuseum		kube2iam		 prometheus-pushgateway
chronograf		kubed			 prometheus-rabbitmq-exporter
cluster-autoscaler	kubedb			 prometheus-redis-exporter
cockroachdb		kube-lego		 prometheus-to-sd
concourse		kube-ops-view		 quassel
consul			kubernetes-dashboard	 rabbitmq
coredns			kuberos			 rabbitmq-ha
coscal

There are various chart repositories available (check out bitnami's as well the helm ones), but this is the git repo used to build the default stable repository. You can see the variety of charts currently there. Lets look inside the `lamp` one.

In [52]:
cd ~/helm-charts/stable/lamp
find .

.
./README.md
./files
./files/httpd
./files/httpd/httpd-vhosts.conf
./files/httpd/httpd-vhosts-socket.conf
./files/httpd/httpd.conf
./files/init
./files/init/init_wp.sh
./files/init/init_wp_db.sh
./files/init/init_clone.sh
./files/init/init_db_clone.sh
./values.yaml
./templates
./templates/ingress.yaml
./templates/NOTES.txt
./templates/_helpers.tpl
./templates/pvc.yaml
./templates/service.yaml
./templates/ingress-services.yaml
./templates/service-sftp.yaml
./templates/deployment.yaml
./templates/configmap-php.yaml
./templates/configmap-httpd.yaml
./templates/configmap-init.yaml
./templates/ingress-www.yaml
./templates/secret.yaml
./Chart.yaml
./examples
./examples/grav.yaml
./examples/wordpress-php-ini.yaml
./examples/joomla.yaml
./examples/wordpress-ingress-ssl.yaml
./examples/wordpress.yaml
./examples/drupal.yaml
./examples/nextcloud.yaml
./examples/owncloud.yaml
./.helmignore


The `./templates` directory is the set of templated YAML files that are used to build the core LAMP setup. To highlight a couple: deployment.yaml set up an Apache server as a Deployment, pvc.yaml requests persistent storage (for the DB to use, called a PersistentVolumeClaim), configmap-init.yaml sets up a config file mounted into the server.

In [53]:
cd ~/helm-charts/stable/lamp
helm install . --values ./examples/wordpress.yaml --name mywordpress

NAME:   mywordpress
LAST DEPLOYED: Wed Oct  3 22:34:41 2018
NAMESPACE: scotlandphp-XXXXX
STATUS: DEPLOYED

RESOURCES:
==> v1/Secret
NAME              TYPE    DATA  AGE
mywordpress-lamp  Opaque  4     0s

==> v1/ConfigMap
NAME                    DATA  AGE
mywordpress-lamp-httpd  3     0s
mywordpress-lamp-php    2     0s

==> v1/PersistentVolumeClaim
NAME              STATUS   VOLUME    CAPACITY  ACCESS MODES  STORAGECLASS  AGE
mywordpress-lamp  Pending  standard  0s

==> v1/Service
NAME              TYPE          CLUSTER-IP   EXTERNAL-IP  PORT(S)                      AGE
mywordpress-lamp  LoadBalancer  10.15.242.8  <pending>    80:30366/TCP,3306:32237/TCP  0s

==> v1beta1/Deployment
NAME              DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
mywordpress-lamp  1        1        1           0          0s

==> v1/Pod(related)
NAME                               READY  STATUS   RESTARTS  AGE
mywordpress-lamp-7b68d9d676-5wbtg  0/3    Pending  0         0s


NOTES:
INIT:
      Please wait f

We just added some custom values in from `./examples/wordpress.yaml` to drop into the LAMP template. We could take a copy of this, fill in our own values and use it as a short per-deployment customization file.

In [54]:
cat ~/helm-charts/stable/lamp/examples/wordpress.yaml

mysql:
  rootPassword: "Root Password here..."
  user: wordpress
  password: "User Password here..."
  database: wordpress

php:
  repository: "wordpress"
  tag: "php7.1-fpm"
  envVars:
  - name: WORDPRESS_DB_HOST
    value: localhost
  - name: WORDPRESS_DB_USER
    value: wordpress
  - name: WORDPRESS_DB_PASSWORD
    value: "User Password here..."
  - name: WORDPRESS_DB_DATABASE
    value: wordpress


In [71]:
kubectl get services

NAME               TYPE           CLUSTER-IP      EXTERNAL-IP    PORT(S)                       AGE
mywordpress-lamp   LoadBalancer   10.15.242.8     35.230.157.0   80:30366/TCP,3306:32237/TCP   54s
tiller-deploy      ClusterIP      10.15.252.217   <none>         44134/TCP                     8m


In [87]:
echo "Internal: " $(kubectl get service mywordpress-lamp --output=jsonpath="{.spec.clusterIP}")
echo "External: http://$(kubectl get service mywordpress-lamp --output=jsonpath="{.status.loadBalancer.ingress[0].ip}")"

Internal:  10.15.242.8
External: http://35.230.157.0


In [88]:
helm delete --purge mywordpress

release "mywordpress" deleted


* Theory of PHP on K8s

A simple app deployed on Kubernetes may look like the following:

[DIAGRAM of SIMPLE APP]

In the PHP case, there are a few caveats...

[DIAGRAM of a PHP APP]

We need the second nginx container to provide HTTP access to PHP-FPM.

# Step 3: Example Laravel App

In [152]:
rm -rf ~/buckram; cd ~
git clone https://gitlab.com/flaxandteal/buckram-starter ~/buckram

Cloning into '/home/jovyan/buckram'...
remote: Enumerating objects: 28788, done.        
remote: Counting objects: 100% (28788/28788), done.        
remote: Compressing objects: 100% (10800/10800), done.        
remote: Total 28788 (delta 17412), reused 28742 (delta 17378)        
Receiving objects: 100% (28788/28788), 8.04 MiB | 9.96 MiB/s, done.
Resolving deltas: 100% (17412/17412), done.


In [153]:
cd ~/buckram/kubernetes
ls

artisan.sh  dummy-values.yaml  sundry	  values.yaml.example
buckram     README.md	       templates


Here we have a few tools for Kubernetes, in particular an example file for setting deployment-specific values.

In [154]:
cd ~/buckram/kubernetes
cat values.yaml.example

# This is a template for the Kubernetes production secrets.
# It is strongly recommended that you create and complete the
# production.yaml file outside of the file tree
laravel-nginx:
  image:
    repository: flaxandteal/buckram-nginx
    tag: stable
  ingress:
    hostname: DNS.DOMAIN
    annotations:
      certmanager.k8s.io/cluster-issuer: letsencrypt-staging
      kubernetes.io/ingress.class: nginx
  laravel:
    serverName: DNS.DOMAIN
laravel-phpfpm:
  image:
    repository: flaxandteal/buckram-phpfpm
    tag: stable
  workerImage:
    repository: flaxandteal/buckram-phpfpm
    tag: stable
  laravel:
    app:
      appKey: base64:xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
      appUrl: "https://DNS.DOMAIN"
      oauthPublicKey: OAUTH_PUBLIC_KEY_B64
      oauthPrivateKey: OAUTH_PRIVATE_KEY_B64
    mail:
      username: "mailuser"
      password: "mailpass"
      host: "smtp.host"
      port: "25"
      fromAddress: "from@example.com"
      fromName: "From Name"
postgresql:
  post

As a quickstart, there is a basic Python tool in the `buckram` repository to autogenerate some values for these (`bookcloth`) - a sample output of this is in `dummy-values.yaml`. We will use this to demonstrate.

We go into the the Buckram Helm chart itself...

In [155]:
cd ~/buckram/kubernetes/buckram
helm dependencies update
helm install . --name=buckram --values=../dummy-values.yaml

Hang tight while we grab the latest from your chart repositories...
...Unable to get an update from the "local" chart repository (http://127.0.0.1:8879/charts):
	Get http://127.0.0.1:8879/charts/index.yaml: dial tcp 127.0.0.1:8879: connect: connection refused
...Successfully got an update from the "stable" chart repository
Update Complete. ⎈Happy Helming!⎈
Saving 6 charts
Deleting outdated charts
2018/10/03 23:35:39 warning: destination for annotations is a table. Ignoring non-table value <nil>
2018/10/03 23:35:39 warning: destination for annotations is a table. Ignoring non-table value <nil>
Error: a release named buckram already exists.
Run: helm ls --all buckram; to check the status of the release
Or run: helm del --purge buckram; to delete it


: 1

In [161]:
cd ~/buckram/kubernetes
git status

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean


As in every type of deployment, we need to have specific custom settings for our own deployment. This answers several questions:

In this approach:

* **Where is our app code stored?**: In the final built images. During continuous integration (or manual building, for smaller scale), pre-prepared base images have the code added in. The built images are then tagged for that code version and pushed to a Docker image registry that the Kubernetes cluster can access.
* **How do we keep our code secure?**: In this example, for simplicity we use the default base images, without custom app code - as such, they are pulled down from the public Docker Hub registry. However, there are private image registries within each cloud platform, e.g. AWS ECR, which Kubernetes can authenticate with to pull down images.
* **What about secrets?**: In this particular approach, we use Kubernetes Secrets, which is simple but only a minimum bar (although, the default implementation is improving). Better practice is to use something like the Kubernetes Vault Operator, but this takes more care to set up.

_None of these are hard and fast_. There are various approaches, with benefits and drawbacks.

In [118]:
kubectl get pods

NAME                                             READY     STATUS      RESTARTS   AGE
buckram-fluentd-79b748589f-bskhx                 1/1       Running     0          17m
buckram-laravel-nginx-6ddb575b85-w86kv           1/1       Running     0          17m
buckram-laravel-phpfpm-1538607360-dh8s4          0/1       Completed   0          2m
buckram-laravel-phpfpm-1538607420-kzdzr          0/1       Completed   0          1m
buckram-laravel-phpfpm-1538607480-mbtgb          0/1       Completed   0          20s
buckram-laravel-phpfpm-5c9698bdd-k67hh           1/1       Running     0          9s
buckram-laravel-phpfpm-worker-7dd798f458-mgz6b   1/1       Running     0          5m
buckram-postgresql-0                             1/1       Running     0          17m
buckram-redis-0                                  1/1       Running     0          17m
fluent-bit-2z978                                 1/1       Running     0          17m
fluent-bit-jkmbr                                 1/1      

Re-run the above until it quietens down ^^^

In [119]:
IP=$(kubectl get service buckram-laravel-nginx --output=jsonpath="{.spec.clusterIP}")
curl http://$IP | head

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2383    0  2383    0     0   1368      0 --:--:--  0:00:01 --:--:--  1367
<!doctype html>
<html lang="en">
    <head>
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta name="viewport" content="width=device-width, initial-scale=1">

        <title>Laravel</title>

        <!-- Fonts -->


([click here](/user/philtweir/notebooks/kubernetes-scotlandphp/visual-notebook.ipynb#Laravel) to get the rendered version)

* Work through the charts

Lets take a look at what just started up:

In [120]:
kubectl get pods

NAME                                             READY     STATUS      RESTARTS   AGE
buckram-fluentd-79b748589f-bskhx                 1/1       Running     0          17m
buckram-laravel-nginx-6ddb575b85-w86kv           1/1       Running     0          17m
buckram-laravel-phpfpm-1538607360-dh8s4          0/1       Completed   0          2m
buckram-laravel-phpfpm-1538607420-kzdzr          0/1       Completed   0          1m
buckram-laravel-phpfpm-1538607480-mbtgb          0/1       Completed   0          45s
buckram-laravel-phpfpm-5c9698bdd-k67hh           1/1       Running     0          34s
buckram-laravel-phpfpm-worker-7dd798f458-mgz6b   1/1       Running     0          6m
buckram-postgresql-0                             1/1       Running     0          17m
buckram-redis-0                                  1/1       Running     0          17m
fluent-bit-2z978                                 1/1       Running     0          17m
fluent-bit-jkmbr                                 1/1     

As above the list of pods is, roughly, the list of processes. There should be some familiar faces there: several PHP-FPM pods, nginx, PostgreSQL and redis.

Alongside them, you can see fluent-bit - it will forward logs from the processes to a single aggregated destination. Between this and redis (queueing, caching, sessions), the PHP-FPM pod does not need to have any state, easing scalability.

By telling PHP to send to stdout by default, we find that the logs from this Pod are indeed the PHP logs:

In [121]:
PHP_POD=$(kubectl get pods --selector=tier=middle -o=name)
kubectl logs $PHP_POD

[03-Oct-2018 22:58:23] NOTICE: fpm is running, pid 1
[03-Oct-2018 22:58:23] NOTICE: ready to handle connections
10.12.1.13 -  03/Oct/2018:22:58:27 +0000 "GET /index.php" 200
10.12.1.13 -  03/Oct/2018:22:58:35 +0000 "GET /index.php" 200


This approach also simplified life for log aggregation, where logs can be gathered from the stdout/stderr of the various Pods.

----
### Break
----

### Gitlab CI

_20 min_ (-> 1:50)

* What is Gitlab CI?

Gitlab continuous integration (CI) provides us with an easy way to go from local development to built images. To follow along, you will need to have [your own Gitlab account or sign up](https://gitlab.com/users/sign_in), or can look through the [details of the pipeline](https://gitlab.com/flaxandteal/buckram-demo/pipelines) of the existing demo.

If you wish to use your own account, then you [should fork](https://gitlab.com/flaxandteal/buckram-demo/forks/new) a copy. You can clone it down here, or on your own machine/VM to make it easier to experiment. If you want to use your fork in this notebook, change the Gitlab URL below to your fork's.

In [122]:
cd ~
rm -rf buckram-demo
git clone https://gitlab.com/phil.weir/buckram-demo

Cloning into 'buckram-demo'...
remote: Enumerating objects: 145, done.        
remote: Counting objects: 100% (145/145), done.        
remote: Compressing objects: 100% (106/106), done.        
remote: Total 145 (delta 22), reused 137 (delta 19)        
Receiving objects: 100% (145/145), 197.99 KiB | 765.00 KiB/s, done.
Resolving deltas: 100% (22/22), done.


This repository has only a couple of minor differences from the usual `composer create-project laravel` output, namely, running composer (after adding behat), a `.gitlab-ci.yml` file and a submodule containing the buckram content, which is in an `infrastructure` subfolder.

In [123]:
cd ~/buckram-demo
git submodule update --init

Submodule 'infrastructure' (https://gitlab.com/flaxandteal/buckram) registered for path 'infrastructure'
Cloning into '/home/jovyan/buckram-demo/infrastructure'...
Submodule path 'infrastructure': checked out '53ab820046e457061b6401c6e1e3ef6bec2e55b5'


Note that our Buckram repository is not necessary for any of this, but is simply a way of bringing together common features, such as CI templates, Helm charts and config generation, which you could do independently. They are useful, accessible examples for a smallish context, but once familiar, you will find a number of features you will wish to add.

In [125]:
cd ~/buckram-demo
git diff $(git rev-list --max-parents=0 HEAD)..HEAD --summary

 create mode 100644 .gitmodules
 create mode 100644 features/bootstrap/FeatureContext.php
 create mode 160000 infrastructure
 delete mode 100644 tests/Feature/ExampleTest.php


First off, we can take a look at the `.gitlab-ci.yml` file:

In [126]:
cat .gitlab-ci.yml

image: docker:latest

variables:
  CONTAINER_NGINX_IMAGE: $CI_REGISTRY_IMAGE:nginx-$CI_PIPELINE_ID
  CONTAINER_NGINX_RELEASE_IMAGE: $CI_REGISTRY_IMAGE:nginx-latest
  CONTAINER_PHPFPM_IMAGE: $CI_REGISTRY_IMAGE:phpfpm-$CI_PIPELINE_ID
  CONTAINER_PHPFPM_RELEASE_IMAGE: $CI_REGISTRY_IMAGE:phpfpm-latest
  COMPOSER_CACHE_DIR: /cache
  DOCKER_DRIVER: overlay

stages:
- composer
- build
- test
- release
- deploy

services:
- docker:dind

before_script:
  - docker login -u gitlab-ci-token -p $CI_BUILD_TOKEN registry.gitlab.com

composer:
  stage: composer
  before_script:
  - echo "Building PHP dependencies"
  image: composer
  script:
  - composer install
  artifacts:
    paths:
    - vendor
    - bootstrap/cache
    - bootstrap/autoload.php
    - composer.lock

build:
  stage: build
  variables:
    GIT_SUBMODULE_STRATEGY: recursive
  script:
  - chown -R 33 storage/logs bootstrap/cache
  - docker build -f infrastructure/containers/nginx/Dockerfile -t $CONTAINER_NGINX_IMAGE .
  - docker build 

In this particular approach, for simplicity, both the nginx and PHP containers end up with the build code - a nicer tactic would be to have any static assets deployed separately, so that nginx only needs to proxy requests to PHP-FPM.

The first few sections are configuration, essentially preparing the environment. The following, `composer`, `build` and `release` steps are the sequence the CI will follow. The first runs composer (it does this inside the official composer Docker image), the second combines these with our pre-prepared base images. For more detail on the PHP-FPM setup used, have a look at `./infrastructure/containers/phpfpm` (while it starts from the official base it has, for instance, ext-redis added in).

In the Buckram repository, there is a template `gitlab-ci.yml` - while it isn't useful for our demo, you will see this has an additional `deploy-dev` section for deploying directly to a Kubernetes cluster. Over the last year or two, Gitlab's direct deployment integration for Kubernetes on Google has come on, so you may want to look at this option too. However, it may be illustrative:

In [128]:
grep -A 100 'deploy_dev:' ~/buckram/gitlab-ci/gitlab-ci.yml

deploy_dev:
  image: google/cloud-sdk:162.0.0
  before_script:
  - kubectl config set-cluster "$CI_PROJECT_ID" --server="$KUBE_URL" --certificate-authority="$KUBE_CA_PEM_FILE"
  - kubectl config set-credentials "$CI_PROJECT_ID" --token="$KUBE_TOKEN"
  - kubectl config set-context "$CI_PROJECT_ID" --cluster="$CI_PROJECT_ID" --user="$CI_PROJECT_ID" --namespace="$KUBE_NAMESPACE"
  - kubectl config use-context "$CI_PROJECT_ID"
  stage: deploy
  script:
  - >-
    kubectl patch cronjob.v2alpha1.batch $CI_ENVIRONMENT_SLUG-laravel-artisan
      -p "{
        \"spec\": {
          \"jobTemplate\": {
            \"spec\": {
              \"template\": {
                \"spec\": {
                  \"containers\": [
                    {
                      \"name\": \"$CI_ENVIRONMENT_SLUG-laravel-scheduler\",
                      \"image\": \"$AWS_ECR_URI:phpfpm-$CI_PIPELINE_ID\"
                    }
                  ]
                }
              }
            }
          }
        }


Modern Kubernetes has Role Based Authorization Control (RBAC), so we can create a user with deployment credentials and Gitlab will ensure our CI user receives them. We use a Docker image with `kubectl` to run these commands. It would be possible do this via `curl` calls also, to the Kubernetes API. A particular benefit of Gitlab-CI is that it can be run internally, or even _on_ Kubernetes, while being usable as a free hosted service to experiment.

It is not the only option, however - Bitbucket now has a similar set-up. Jenkins, CircleCI and others can provide these types of pipeline also.

The rest of the CI steps show some of the Docker commands that could be run locally, if you wanted to get the hang of Docker in a manual way, building images and pushing them.

The test step lets us run `behat` and `phptest`, by default, when any branch is pushed. Gitlab will notify you of any errors, and can trigger a merge based on its outcome, for instance. Code coverage and linting could similarly be added here.

The magic of Gitlab-CI is that it will start working as soon as receives a commit with a valid `.gitlab-ci.yml` file in the root directory (note the first dot).

In [129]:
grep -C 5 ' Laravel' resources/views/welcome.blade.php

                </div>
            @endif

            <div class="content">
                <div class="title m-b-md">
                    LaravelTest
                </div>

                <div class="links">
                    <a href="https://laravel.com/docs">Documentation</a>
                    <a href="https://laracasts.com">Laracasts</a>


If you have taken a fork, then you can try the following (otherwise, keep an eye on my fork's [pipeline's page](https://gitlab.com/flaxandteal/buckram-demo/pipelines)):

https://gitlab.com/USERNAME/buckram-demo/blob/master/resources/views/welcome.blade.php

(change the USERNAME to your own). Edit the page (Edit button on upper right), perhaps changing the word Laravel on line 82 to something else. This is equivalent to making a manual change and pushing.

On the Pipelines page (under CI/CD on the left-hand menu), you will see the progress of the building through the stages. Gitlab uses its own container registry to manage the images back and forward, and some handy free CI runners. Running your own CI runner is easy, and generally much faster per run, not to mention allowing you to control the process. Another key benefit is that you can also cache composer downloads and Docker images between steps and runs - although many prefer to take the extra time to pull fresh from the package repository on each build anyway.

A note on frontend: if using little frontend code or a tightly-integrated frontend framework in the Laravel codebase, then this may be sufficient. Most of our work is with API-based backends and self-contained VueJS single-page applications. Our approach is to run the necessary CI steps on the frontend repository, producing a JS/CSS tarball that is released to S3, and trigger a new backend pipeline run from the final frontend step, to pull down and build a fresh final set of Docker images.

The following command manually updates the PHP-FPM image to point to the newly build Gitlab one (NB: only the web-serving one, not the queue worker or cronjob). You should swap the `phil.weir` and number at the end for your username and pipeline number, respectively.

_Make sure you don't confuse the pipeline and job numbers_, the pipeline number will be shown on the Pipelines page, usually starting with a hash.

In [169]:
kubectl set image deployment/buckram-laravel-phpfpm laravel-phpfpm=registry.gitlab.com/phil.weir/buckram-demo:phpfpm-31946481

deployment.extensions/buckram-laravel-phpfpm image updated


In [177]:
kubectl get pods --selector=app=buckram-laravel-phpfpm

NAME                                             READY     STATUS    RESTARTS   AGE
buckram-laravel-phpfpm-66568bdc97-pvv58          1/1       Running   0          13s
buckram-laravel-phpfpm-worker-7dd798f458-mgz6b   1/1       Running   0          1h


When the above command shows "Status: running", then try the Laravel call in the [other notebook](/user/philtweir/notebooks/kubernetes-scotlandphp/visual-notebook.ipynb#Laravel) again - you should see the altered welcome page from your newly built image!

Kubernetes provides some useful tools for managing roll-out. For instance:

In [134]:
kubectl rollout history deployments

deployments "buckram-fluentd"
REVISION  CHANGE-CAUSE
1         <none>

deployments "buckram-laravel-nginx"
REVISION  CHANGE-CAUSE
1         <none>

deployments "buckram-laravel-phpfpm"
REVISION  CHANGE-CAUSE
1         <none>
2         <none>
3         <none>

deployments "buckram-laravel-phpfpm-worker"
REVISION  CHANGE-CAUSE
1         <none>
2         <none>

deployments "tiller-deploy"
REVISION  CHANGE-CAUSE
1         <none>
2         <none>



Here you can see how many revisions have been made to your deployments - you should see 3, one for each of fluentd, nginx and phpfpm-worker, all with 1 revision, and a fourth for phpfpm, with an additional revision (assuming you correctly deleted the earlier experiments!). This additional revision represents the action of changing the Docker image used, above.

You can roll back deployments, you can require a certain minimum number of live pods at any time during the rolling update, or even decide where new Pods are scheduled on the underlying VMs/systems (termed Nodes). Kubernetes has concepts of liveness and readiness probes, so for some types of deployment, it can spot when Pods are functional or not. In addition, this allows it to undertake one of the most fundamental aspects of orchestration, replacing Pods when they become unresponsive or die.

### Artisan and Jobs
(moved from end?)

_20 min_ (-> 2:10)

We will address a few of the key features of Laravel, and PHP frameworks in general, here.

* Worker process & Redis

It's already been pointed out that there is an artisan Pod running - phpfpm-worker. For those unfamiliar, `artisan` is Laravel's command-line tool, providing console control for one-off tasks, but also entrypoints for worker processes and the task scheduler.

In normal usage, getting an artisan queue worker process running involves:

    php artisan queue:work

You may wish to use this to, for instance, submit emails to a sending service or call batch services.

In this context, it becomes part of the `development-artisan.yaml` Chart file (see the arrays around line 25):

In [137]:
cat ~/buckram/kubernetes/buckram/subcharts/laravel-phpfpm/templates/deployment-artisan.yaml | nl

     1	apiVersion: extensions/v1beta1
     2	kind: Deployment
     3	metadata:
     4	  name: {{ template "fullname" . }}-worker
     5	  labels:
     6	    chart: "{{ .Chart.Name }}-{{ .Chart.Version | replace "+" "_" }}"
     7	    release: "{{ .Release.Name }}"
     8	    heritage: "{{ .Release.Service }}"
     9	spec:
    10	  replicas: {{ .Values.replicaCount }}
    11	  template:
    12	    metadata:
    13	      labels:
    14	        app: {{ template "fullname" . }}
    15	        tier: "backend"
    16	    spec:
    17	{{- if .Values.workerImage.secrets }}
    18	      imagePullSecrets:
    19	      - name: {{ .Values.workerImage.secrets }}
    20	{{- end }}
    21	      containers:
    22	      - name: {{ .Chart.Name }}-worker
    23	        command:
    24	        - php
    25	        - /var/www/app/artisan
    26	        args:
    27	        - queue:work
    28	{{- if .Values.laravel.queue.tries }}
    29	        - --tries={{ .Values.laravel.queue.tries }}
    30	{{- end }}

You can contrast the in situ version, by pulling down the definition of the live pod, created from this template.

In [167]:
WORKER_POD=$(kubectl get pods --selector=tier=backend -o=name)
kubectl get $WORKER_POD -o=yaml

apiVersion: v1
kind: Pod
metadata:
  creationTimestamp: 2018-10-03T22:52:36Z
  generateName: buckram-laravel-phpfpm-worker-7dd798f458-
  labels:
    app: buckram-laravel-phpfpm
    pod-template-hash: "3883549014"
    tier: backend
  name: buckram-laravel-phpfpm-worker-7dd798f458-mgz6b
  namespace: scotlandphp-XXXXX
  ownerReferences:
  - apiVersion: extensions/v1beta1
    blockOwnerDeletion: true
    controller: true
    kind: ReplicaSet
    name: buckram-laravel-phpfpm-worker-7dd798f458
    uid: 058d907e-c75f-11e8-b213-42010a9a007a
  resourceVersion: "15726"
  selfLink: /api/v1/namespaces/scotlandphp-XXXXX/pods/buckram-laravel-phpfpm-worker-7dd798f458-mgz6b
  uid: 0592eb2a-c75f-11e8-b213-42010a9a007a
spec:
  containers:
  - args:
    - queue:work
    - --tries=3
    command:
    - php
    - /var/www/app/artisan
    env:
    - name: APP_KEY
      valueFrom:
        secretKeyRef:
          key: app-key
          name: buckram-laravel-phpfpm-env
    - name: MAIL_PASSWORD
      valueFrom:

As running Redis in the cluster is an easy starting point, we use it to handle the queues (Laravel has Redis as an in-built option):

In [139]:
kubectl logs $(kubectl get pods --selector=app=buckram-redis -o=name)

nami    INFO  Initializing redis
redis   INFO 
redis   INFO  ########################################################################
redis   INFO   Installation parameters for redis:
redis   INFO     Password: **********
redis   INFO   (Passwords are not shown for security reasons)
redis   INFO  ########################################################################
redis   INFO 
nami    INFO  redis successfully initialized
Starting application ...

  *** Welcome to the redis image ***
  *** Brought to you by Bitnami ***
  *** More information: https://github.com/bitnami/bitnami-docker-redis ***
  *** Issues: https://github.com/bitnami/bitnami-docker-redis/issues ***

 |    `-._`-._        _.-'_.-'    |                                  
  `-._    `-._`-.__.-'_.-'    _.-'                                   
      `-._    `-.__.-'    _.-'                                       
          `-._        _.-'                                           
              `-.__.-'                   

### One-off Jobs

* Cronjobs

To make scheduled tasks work, we use the Kubernetes concept of Cronjobs.

In [140]:
kubectl get cronjobs

NAME                     SCHEDULE    SUSPEND   ACTIVE    LAST SCHEDULE   AGE
buckram-laravel-phpfpm   * * * * *   False     0         32s             25m


You will see this uses a similar format to traditional cron schedules. As with the normal Laravel approach, we set this to run an artisan job every minute (`php artisan schedule:run`), and Laravel's scheduler will decide whether any of the PHP-defined tasks are due to get kicked off.

This is quite configurable - we can say how many completed Pods we want to keep, for diagnostics, and how many failed ones, how long we give them to start up, and how many attempts each will get.

* Running artisan

This is part of a broader concept of Jobs. These are requests to schedule a one-off Pod to do some task or other. Kubernetes' Cronjobs are really just creating a "Job" each minute from a template. Jobs in general provide a reasonable way to run one-off artisan tasks also.

At present, we use a barebones script to simplify this process - running it creates a fresh new Job (from the template as the Cronjob, as it happens), and sets the internal artisan arguments to whichever command line flags we pass.

In [141]:
cd ~/buckram/kubernetes
./artisan.sh route:list


job.batch/laravel-job-1538607998 created
Flag --show-all has been deprecated, will be removed in an upcoming release
Found pod: laravel-job-1538607998-n94kw
Error from server (BadRequest): container "laravel-phpfpm-scheduler" in pod "laravel-job-1538607998-n94kw" is waiting to start: ContainerCreating
Error from server (BadRequest): container "laravel-phpfpm-scheduler" in pod "laravel-job-1538607998-n94kw" is waiting to start: ContainerCreating
+--------+----------+----------+------+---------+--------------+
| Domain | Method   | URI      | Name | Action  | Middleware   |
+--------+----------+----------+------+---------+--------------+
|        | GET|HEAD | /        |      | Closure | web          |
|        | GET|HEAD | api/user |      | Closure | api,auth:api |
+--------+----------+----------+------+---------+--------------+


You will get a few Errors as it starts up (ending "ContainerCreating"), but it should follow the Pod's logs once the Pod has started. Note that this approach is non-interactive (in fact, asynchronous).

In [142]:
kubectl get jobs

NAME                                DESIRED   SUCCESSFUL   AGE
buckram-laravel-phpfpm-1538607840   1         1            2m
buckram-laravel-phpfpm-1538607900   1         1            1m
buckram-laravel-phpfpm-1538607960   1         1            42s
laravel-job-1538607998              1         1            9s


Above you should see a range of Jobs that have been run, including your manual one as "laravel-job" and a timestamp.

### Managing Environments

_20 min_ (-> 2:30)

As in any other setting, you will want to have more than one app platform running at once: usually for development, staging and production; or perhaps blue/green deployments.

Gitlab provide tools to help manage separate environments, along with Kubernetes - some of that is quite tied to Google Kubernetes Engine, but some is platform agnostic.

Taking a separate Helm `values.yaml` file may be sufficient to give, e.g. a dev and staging cluster. You may be happy enough to have these both on the same Kubernetes cluster, treating it as an infrastructure provider, as long as they are namespaced.

Kubernetes does provide namespacing - the divisions are being hardened at each version, and it is possible to apply resource usage policies, user role policies and network segmentation.

In [144]:
kubectl get pods -n kube-system

NAME                                                      READY     STATUS    RESTARTS   AGE
event-exporter-v0.2.1-5f5b89fcc8-7lzcn                    2/2       Running   0          2h
fluentd-gcp-scaler-7c5db745fc-l8srn                       1/1       Running   0          2h
fluentd-gcp-v3.1.0-8pssr                                  2/2       Running   0          2h
fluentd-gcp-v3.1.0-hftfn                                  2/2       Running   0          2h
fluentd-gcp-v3.1.0-vcj6p                                  2/2       Running   0          2h
heapster-v1.5.3-6795f54f76-8jhlg                          3/3       Running   0          2h
kube-dns-788979dc8f-khnmc                                 4/4       Running   0          2h
kube-dns-788979dc8f-vrcrd                                 4/4       Running   0          2h
kube-dns-autoscaler-79b4b844b9-nwd9t                      1/1       Running   0          2h
kube-proxy-gke-scotlandphp-3-default-pool-32254843-3rbk   1/1       Running   0

You can see here that there are a number of Pods doing more fundamental things than nginx - providing internal DNS, managing Helm's requests, handling Kubernetes API calls, running controller loops. These are all in the `kube-system` namespace by default.

# Contexts

Kubectl works on a concept of users, contexts and clusters. You might not have spotted yet, but Kubernetes does not have a concept of user accounts, per se. It does have a concept of authentication, which can be by certificates, tokens, OpenID Connect, for example, and authorization Roles, which are bound to a username matching a RoleBinding rule when it successfully authenticates. However, no separate "User" object exists.

Contexts allow you to specify a user, a namespace and a cluster to work with by default. Switching contexts allows you to easily manage multiple clusters and namespaces.

In [162]:
kubectl config get-contexts

CURRENT   NAME              CLUSTER           AUTHINFO       NAMESPACE
*         jupyter-cluster   jupyter-cluster   jupyter-user   scotlandphp-XXXXX


In the `~/buckram/docs` folder is a script, `update_cluster_to_ci_build_ref.sh`, to update one context from another - this can be very handy for manual promotion, where you want to have specific controls in place to control access to the update workflow.

### Health, Logging and Scaling

_30 min_ (-> 3:00)

* Monitoring

There are a number of monitoring solutions available. CoreOS, one of the driver organizations behind containerization, have provided a [Kubernetes Operator for Prometheus](https://github.com/coreos/prometheus-operator) - Operators are a relative recent, powerful tool for making Kubernetes more extensible, and allowing automated control loops, for example, to take care of dynamically-defined types of object.

Installing this operator, which can be done with `helm`, adds in several new types of Kubernetes object: PrometheusRule and AlertingRule being two examples. The operator can then keep a cluster-hosted Prometheus server dynamically changing its configuration as those are defined.

An additional Helm chart, [kube-prometheus](https://github.com/coreos/prometheus-operator/tree/master/helm/kube-prometheus) builds on this to provide live feeds of Pod, Service, Node resource usage, and alerting based off that. We have implemented a basic monitoring chart that adds support for Laravel failed job and exception tracking and alerts. While some of this can be done through the framework, this ensures that metrics and alerting can be managed centrally, and that individual, scaled out web-serving or queue processes are not responsible for contacting external APIs - rather they feed it back internally to be grouped, rate-managed, etc. in a standard way with the rest of the infrastructure.

* Log aggregation (fluentbit, fluentd)

As part of the deployment, we create fluent-bit DaemonSets - these are essentially Deployments that run one Pod on each Node (VM). Gathering logs is a perfect application for them. Fluent-bit is a reimplementation in C of a lot of Fluentd's functionality. Fluent-bit can gather logs from the containers' stdout/stderr and send these to Fluentd, or to a number of other aggregators, such as Elasticsearch.

In [148]:
kubectl get pods

NAME                                             READY     STATUS      RESTARTS   AGE
buckram-fluentd-79b748589f-bskhx                 1/1       Running     0          48m
buckram-laravel-nginx-6ddb575b85-w86kv           1/1       Running     0          48m
buckram-laravel-phpfpm-1538609220-khvn2          0/1       Completed   0          2m
buckram-laravel-phpfpm-1538609280-xz5kz          0/1       Completed   0          1m
buckram-laravel-phpfpm-1538609340-vjcxg          0/1       Completed   0          58s
buckram-laravel-phpfpm-1538609400-x6cb8          0/1       Completed   0          7s
buckram-laravel-phpfpm-587d4c6fd8-cm59n          1/1       Running     0          29m
buckram-laravel-phpfpm-worker-7dd798f458-mgz6b   1/1       Running     0          37m
buckram-postgresql-0                             1/1       Running     0          48m
buckram-redis-0                                  1/1       Running     0          48m
fluent-bit-2z978                                 1/1     

In [160]:
helm list

NAME   	REVISION	UPDATED                 	STATUS  	CHART        	NAMESPACE        
buckram	2       	Wed Oct  3 23:35:55 2018	DEPLOYED	buckram-0.0.2	scotlandphp-XXXXX


In [168]:
kubectl logs $(kubectl get pods --selector=app=buckram-fluentd -o=name)

chown: /fluentd/etc/fluent.conf: Read-only file system
2018-10-03 23:54:23 +0000 [info]: parsing config file is succeeded path="/fluentd/etc/fluent.conf"
2018-10-03 23:54:23 +0000 [info]: using configuration file: <ROOT>
  <source>
    @type forward
    @id input1
    @label @mainstream
    port 24284
  </source>
  <label @mainstream>
    <match kube.var.log.containers.**_kube-system_**>
      @type null
    </match>
    <match>
      @type file
      path "/fluentd/log/myapp"
      time_slice_format %Y%m%d%H
      <buffer time>
        timekey 3600
        path /fluentd/log/myapp
      </buffer>
    </match>
  </label>
</ROOT>
2018-10-03 23:54:23 +0000 [info]: starting fluentd-1.2.5 pid=6 ruby="2.4.4"
2018-10-03 23:54:23 +0000 [info]: spawn command to main:  cmdline=["/usr/bin/ruby", "-Eascii-8bit:ascii-8bit", "/usr/bin/fluentd", "-c", "/fluentd/etc/fluent.conf", "-p", "/fluentd/plugins", "--under-supervisor"]
2018-10-03 23:54:24 +0000 [info]: gem 'fluentd' version '1.2.5'
2018-10-03 

* Autoscaling

It is possible to scale the number of deployed Pods automatically, using a HorizontalPodAutoscaler object, which responds to [predefined or custom metrics](https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale/). This is simplest to set up for CPU requests, but can work off, for example, [Prometheus metrics](https://github.com/directxman12/k8s-prometheus-adapter).

On the other hand, [Cluster Autoscaling](https://github.com/kubernetes/autoscaler) (where the number of Nodes changes) is cloud provider dependent. This is well-established especially for GKE, but also available for other Kubernetes providers.

# Minikube

(time permitting)

```
minikube start
[wait]
kubectl get nodes
```

This will show you one node - the VM that is running Kubernetes, and all that is on it. In normal usage, you would have a larger set of nodes - many setups will have a pool of nodes for masters, usually high in CPU resources, and a pool of nodes, perhaps smaller and more plentiful for granular horizontal scaling.

# Next steps

I hope you enjoyed this workshop and am looking forward to feedback!

* Follow-up course